In [1]:
from route_shape_process_scripts import *
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import numpy as np
import time

# import gtfs from 01_gtfs_transform notebook output

In [2]:
full_routes_gtfs = pd.read_csv("gtfs_routes_2018-08-15_2018-12-12.csv")
full_shapes_gtfs = pd.read_csv("gtfs_shapes_2018-08-15_2018-12-12.csv")
full_trips_gtfs = pd.read_csv("gtfs_trips_2018-08-15_2018-12-12.csv")
full_trip_stop_schedule = pd.read_csv("gtfs_2018-08-15_2018-12-12.csv")

/Users/benjaminmalnor/anaconda2/envs/geopy36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
full_trip_stop_schedule.groupby(['start_gtfs_date','end_gtfs_date']).agg({'trip_id':'count'})

,,trip_id
start_gtfs_date,end_gtfs_date,
2018-08-15,2018-09-23,2062854
2018-09-24,2018-09-24,1038258
2018-09-25,2018-11-01,1038258
2018-11-02,2018-11-09,1033065
2018-11-10,2018-12-11,1033163
2018-12-12,2019-01-07,1754837


In [4]:
full_trip_stop_schedule_dict = {}
for name, group in full_trip_stop_schedule.groupby(['start_gtfs_date','end_gtfs_date']):
    full_trip_stop_schedule_dict[name] = group

# analyze September --> November

## get route of interest

In [5]:
get_select_routeid_name(full_routes_gtfs, ['E Line'])[2]

{'E Line': 102615}

# make sure the fuction below is pointing to your local folder with the position_yearmonth.h5 data

In [6]:
def get_positions_months_route_id(month_list, route_id):
    '''
    month_list = ['201809', '201810', '201811'] need to point to
    the right folder holding your h5 files
    '''
    for i, position_date in enumerate(month_list):
        if i == 0:
            positions = pd.read_hdf("/Users/benjaminmalnor/sandbox/bus350/data/intermed/positions_{}.h5".format(position_date))
            print(position_date,len(positions), positions.columns)
            route_positions = positions[positions['route_id']==route_id].copy()
            #del [positions_201809]
            full_route_positions = route_positions.copy()
        else:
            positions = pd.read_hdf("/Users/benjaminmalnor/sandbox/bus350/data/intermed/positions_{}.h5".format(position_date))
            print(position_date,len(positions), positions.columns)
            route_positions = positions[positions['route_id']==route_id].copy()
            #del [positions_201809]
            full_route_positions = full_route_positions.append(route_positions)
    return full_route_positions

# get subset of position files for just one route_id

In [7]:
month_list = ['201809', '201810', '201811']
input_dict = {'route_id':102615}
full_route_positions = get_positions_months_route_id(month_list, input_dict['route_id'])

201809 9263521 Index(['route_id', 'timestamp', 'trip_id', 'vehicle_id', 'vehicle_lat',
       'vehicle_long', 'time_utc', 'time_pct'],
      dtype='object')
201810 10655967 Index(['route_id', 'timestamp', 'trip_id', 'vehicle_id', 'vehicle_lat',
       'vehicle_long', 'time_utc', 'time_pct'],
      dtype='object')
201811 10175465 Index(['route_id', 'timestamp', 'trip_id', 'vehicle_id', 'vehicle_lat',
       'vehicle_long', 'time_utc', 'time_pct'],
      dtype='object')


# add time index columns

In [8]:
full_route_positions = convert_index_to_pct(full_route_positions)
full_route_positions = add_time_index_columns(full_route_positions)

# need to print out shape_ids - find the shape to process
***
# TODO - get direction id get edges for all shapes but keep shape_id column for filtering

In [9]:
for name, group in full_trip_stop_schedule.groupby(['start_gtfs_date','end_gtfs_date']):
    print(name)
    shape_trips_df = group[group['route_id'] == input_dict['route_id']].copy()
    print(shape_trips_df.groupby(['route_id','shape_id', 'direction_id','trip_headsign']).agg({'shape_id':'count'}))

('2018-08-15', '2018-09-23')
                                                                 shape_id
route_id shape_id direction_id trip_headsign                             
102615   11675003 0            Aurora Village Transit Center        19285
         20675007 1            Downtown Seattle                     19250
         20675030 1            Stadium Station Downtown Seattle       144
('2018-09-24', '2018-09-24')
                                                                 shape_id
route_id shape_id direction_id trip_headsign                             
102615   11675003 0            Aurora Village Transit Center         9730
         20675007 1            Downtown Seattle                      9765
         20675030 1            Stadium Station Downtown Seattle        72
('2018-09-25', '2018-11-01')
                                                                 shape_id
route_id shape_id direction_id trip_headsign                             
102615   11675003 0      

# pick a shape_id out of the above options

In [10]:
input_dict['shape_id'] = 11675003

# make route_vertex_geo_dict and G_dict

In [11]:
route_vertex_geo_dict = {}
G_dict = {}
for name, group in full_shapes_gtfs.groupby(
                ['start_gtfs_date','end_gtfs_date']):
    print(name)
    route_vertex_geo_dict[name], G_dict[name] = get_route_vertex_graph(group, input_dict['shape_id'])

('2018-08-15', '2018-09-23')
('2018-09-24', '2018-09-24')
('2018-09-25', '2018-11-01')
('2018-11-02', '2018-11-09')
('2018-11-10', '2018-12-11')
('2018-12-12', '2019-01-07')


# join position table with trip gtfs information

In [12]:
full_trips_gtfs.head(3)

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,peak_flag,fare_id,start_gtfs_date,end_gtfs_date
0,100227,19845,34734386,Downtown Seattle,LOCAL,1,5045281,20047002,0,101.0,2018-08-15,2018-09-23
1,100227,19845,34734387,Downtown Seattle,LOCAL,1,5045281,20047002,0,101.0,2018-08-15,2018-09-23
2,100227,19845,34734388,Downtown Seattle,LOCAL,1,5045281,20047002,0,101.0,2018-08-15,2018-09-23


In [13]:
positions_w_trips = {}
for name, group in full_trips_gtfs.groupby(['start_gtfs_date','end_gtfs_date']):
    print(name)
    positions_w_trips[name] = join_positions_with_gtfs_trips(full_route_positions, group, name[0], name[1])

('2018-08-15', '2018-09-23')
('2018-09-24', '2018-09-24')
('2018-09-25', '2018-11-01')
('2018-11-02', '2018-11-09')
('2018-11-10', '2018-12-11')
('2018-12-12', '2019-01-07')


In [14]:
positions_w_trips[('2018-08-15', '2018-09-23')].head(3)

,route_id,timestamp,trip_id,vehicle_id,vehicle_lat,vehicle_long,time_utc,time_pct,day,year,month,hour,dow,direction_id,shape_id
0,102615,1535759940,39512531,6218,47.682484,-122.344543,2018-08-31 23:59:00,2018-08-31 16:59:00-07:00,31,2018,8,16,4,1,20675007
1,102615,1535759947,39512518,6046,47.723824,-122.344978,2018-08-31 23:59:07,2018-08-31 16:59:07-07:00,31,2018,8,16,4,0,11675003
2,102615,1535759960,39512519,6064,47.614983,-122.345268,2018-08-31 23:59:20,2018-08-31 16:59:20-07:00,31,2018,8,16,4,0,11675003


# get unique_trip_list_dict and positions_w_trips_geo_dict

In [15]:
unique_trip_list_dict = {}
positions_w_trips_geo_dict = {}

for gtfs_groups in positions_w_trips.keys():
    print(gtfs_groups)
    unique_trip_list_dict[gtfs_groups], positions_w_trips_geo_dict[gtfs_groups] = get_trip_from_shape_id(input_dict['shape_id'], 
                                                                                                        positions_w_trips[gtfs_groups])


('2018-08-15', '2018-09-23')
('2018-09-24', '2018-09-24')
('2018-09-25', '2018-11-01')
('2018-11-02', '2018-11-09')
('2018-11-10', '2018-12-11')
('2018-12-12', '2019-01-07')
no trips


In [16]:
positions_w_trips_geo_dict[('2018-08-15', '2018-09-23')].head(3)

,route_id,timestamp,trip_id,vehicle_id,vehicle_lat,vehicle_long,time_utc,time_pct,day,year,month,hour,dow,direction_id,shape_id,month_day_trip_veh,geometry
1,102615,1535759947,39512518,6046,47.723824,-122.344978,2018-08-31 23:59:07,2018-08-31 16:59:07-07:00,31,2018,8,16,4,0,11675003,2018_8_31_39512518_6046,POINT (-122.3449783325195 47.72382354736328)
2,102615,1535759960,39512519,6064,47.614983,-122.345268,2018-08-31 23:59:20,2018-08-31 16:59:20-07:00,31,2018,8,16,4,0,11675003,2018_8_31_39512519_6064,POINT (-122.3452682495117 47.61498260498047)
4,102615,1535759972,39512951,2667,47.664989,-122.347260,2018-08-31 23:59:32,2018-08-31 16:59:32-07:00,31,2018,8,16,4,0,11675003,2018_8_31_39512951_2667,POINT (-122.3472595214844 47.66498947143555)


In [17]:
route_vertex_geo_dict[('2018-08-15', '2018-09-23')].head()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,start_gtfs_date,end_gtfs_date,geometry
130945,11675003,47.601677,-122.330170,1,0.0,2018-08-15,2018-09-23,POINT (-122.33017 47.6016769)
130946,11675003,47.601719,-122.330269,2,30.0,2018-08-15,2018-09-23,POINT (-122.330269 47.6017189)
130947,11675003,47.601788,-122.330429,3,78.6,2018-08-15,2018-09-23,POINT (-122.330429 47.6017876)
130948,11675003,47.601826,-122.330513,4,100.4,2018-08-15,2018-09-23,POINT (-122.330513 47.60182570000001)
130949,11675003,47.601872,-122.330566,5,123.0,2018-08-15,2018-09-23,POINT (-122.330566 47.6018715)


# save the route shape locally if you want to view it on a map

In [18]:
route_json = route_vertex_geo_dict[('2018-08-15', '2018-09-23')].to_json()
file_name = "route_{}_shape_{}.geojson".format(input_dict['route_id'], input_dict['shape_id'])
with open(file_name, 'w') as f:
    f.write(route_json)

In [19]:
def get_point_coords(row, route_vertex_geo):
    if isinstance(row['geometry'], float):
        return ''
    else:
        return get_close_node(row['geometry'].coords[:][0], route_vertex_geo)

# try parallel below - I don't think the parallel function will work on a Windows computer (I've had trouble in the past). It should work fine on a Mac.

In [20]:
import multiprocessing

In [21]:
def get_close_node_parallel(df_group, route_vertex_geo):
    '''
    '''
    df_group['close_node_tuple'] = df_group.apply(get_point_coords, route_vertex_geo=route_vertex_geo, axis=1).copy()
    return df_group

In [22]:
def full_step_process():
    n_pools = multiprocessing.cpu_count() - 1
    pool = multiprocessing.Pool(n_pools)
    positions_w_near_node_dict = {}
    for gtfs_group in positions_w_trips.keys():
        print("starting {}".format(gtfs_group))
        if positions_w_trips_geo_dict[gtfs_group].empty:
            pass
        else:
            grouped = positions_w_trips_geo_dict[gtfs_group].groupby('month_day_trip_veh')
            route_vertex_geo = route_vertex_geo_dict[gtfs_group]
            trip_group_tuple_list = []
            for name, group in grouped:
                trip_group_tuple_list.append((group, route_vertex_geo))
            positions_w_near_node_df = pd.concat(pool.starmap(get_close_node_parallel, trip_group_tuple_list))
            positions_w_near_node_dict[gtfs_group] = positions_w_near_node_df
    return positions_w_near_node_dict

# the below takes ~2-15 minutes on my computer depending on the shape/number of trips

In [23]:
if __name__ == "__main__":
    start = time.time()
    positions_w_near_node_dict = full_step_process()
    end = time.time()
    print(end - start)

starting ('2018-08-15', '2018-09-23')
starting ('2018-09-24', '2018-09-24')
starting ('2018-09-25', '2018-11-01')
starting ('2018-11-02', '2018-11-09')
starting ('2018-11-10', '2018-12-11')
starting ('2018-12-12', '2019-01-07')
1023.4023621082306


In [24]:
def unpack_near_node_column(positions_w_near_node_dict):
    for gtfs_group in positions_w_near_node_dict.keys():
        positions_w_near_node_dict[gtfs_group]['near_node_pt'] = positions_w_near_node_dict[gtfs_group].apply(lambda x: x['close_node_tuple'][0], axis=1)
        positions_w_near_node_dict[gtfs_group]['shape_pt_sequence'] = positions_w_near_node_dict[gtfs_group].apply(lambda x: x['close_node_tuple'][1], axis=1)
        positions_w_near_node_dict[gtfs_group]['dist_to_nearest_route_pt'] = positions_w_near_node_dict[gtfs_group].apply(lambda x: x['close_node_tuple'][2], axis=1)
        positions_w_near_node_dict[gtfs_group].drop('close_node_tuple', axis=1, inplace=True)
        positions_w_near_node_dict[gtfs_group].drop_duplicates(['month_day_trip_veh','shape_pt_sequence'], keep='last', inplace=True)
    return positions_w_near_node_dict

In [25]:
unpacked_positions_w_near_node_dict = unpack_near_node_column(positions_w_near_node_dict)

In [26]:
unpacked_positions_w_near_node_dict[('2018-08-15', '2018-09-23')].head(3)

,route_id,timestamp,trip_id,vehicle_id,vehicle_lat,vehicle_long,time_utc,time_pct,day,year,month,hour,dow,direction_id,shape_id,month_day_trip_veh,geometry,near_node_pt,shape_pt_sequence,dist_to_nearest_route_pt
436,102615,1535761686,39512381,6117,47.604149,-122.332672,2018-09-01 00:28:06,2018-08-31 17:28:06-07:00,31,2018,8,17,4,0,11675003,2018_8_31_39512381_6117,POINT (-122.3326721191406 47.6041488647461),"(-122.332672, 47.6041527)",11,0.000004
528,102615,1535762046,39512381,6117,47.607029,-122.335304,2018-09-01 00:34:06,2018-08-31 17:34:06-07:00,31,2018,8,17,4,0,11675003,2018_8_31_39512381_6117,POINT (-122.3353042602539 47.60702896118164),"(-122.335297, 47.6070137)",16,0.000017
592,102615,1535762317,39512381,6117,47.610466,-122.338448,2018-09-01 00:38:37,2018-08-31 17:38:37-07:00,31,2018,8,17,4,0,11675003,2018_8_31_39512381_6117,POINT (-122.3384475708008 47.61046600341797),"(-122.338432, 47.6104507)",21,0.000022


# merge all dictionaries into one

In [27]:
for idx, dict_group in enumerate(unpacked_positions_w_near_node_dict.keys()):
    print(dict_group)
    if idx == 0:
        unpacked_positions_full = unpacked_positions_w_near_node_dict[dict_group].copy()
    else:
        unpacked_positions_full = unpacked_positions_full.append(unpacked_positions_w_near_node_dict[dict_group])

('2018-08-15', '2018-09-23')
('2018-09-24', '2018-09-24')
('2018-09-25', '2018-11-01')
('2018-11-02', '2018-11-09')
('2018-11-10', '2018-12-11')


# export

In [28]:
unpacked_positions_full.to_csv('route_{}_shape_{}_raw_w_nearest_2018-08-15_2018-12-11.csv'.format(
                                        input_dict['route_id'], input_dict['shape_id']), index=False)

# get start of schedue time (join in gtfs) then calculate just timing to shape distance

In [29]:
def datetime_transform_df(df):
    '''
    '''
    df['time_pct'] = df['time_pct'].apply(pd.to_datetime)
    df.set_index('time_pct', inplace=True)
    df.sort_index(inplace=True)
    df = df.tz_localize('UTC')
    df = df.tz_convert('US/Pacific')
    
    return df

In [30]:
def join_tripstart_unpacked(unpacked_positions_w_near_node_df, trip_stops_w_name_route):
    '''
    '''
    normal_hours = [6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    #drop hours near '0' because the hour comparison gets messed up - gtfs has hours > 23 :(
    unpacked_positions_w_near_node_df = unpacked_positions_w_near_node_df[unpacked_positions_w_near_node_df.hour.isin(normal_hours)]
    
    if 'trip_stops_w_name_route' in trip_stops_w_name_route.columns:
        trip_stops_w_name_route.drop('trip_stops_w_name_route', axis=1, inplace=True)
    position_w_node_schedule = unpacked_positions_w_near_node_df.merge(trip_stops_w_name_route,how='left',
                                                    left_on=['trip_id','route_id','shape_id','shape_pt_sequence'], 
                                                    right_on=['trip_id','route_id','shape_id','stop_sequence'])
    
    position_w_node_schedule = position_w_node_schedule[position_w_node_schedule['stop_name'].notnull()]
    
    
    
    position_w_node_schedule = datetime_transform_df(position_w_node_schedule)
    
    position_w_node_schedule.drop_duplicates(['month_day_trip_veh','shape_pt_sequence'], keep='last', inplace=True)

    position_w_node_schedule['trip_start_time'] = position_w_node_schedule['trip_start_time'].apply(pd.to_datetime)
        
    #take the time at every stop and subtract the scheduled start time
    position_w_node_schedule['time_from_scheduled_start'] = (((position_w_node_schedule.index.hour)*60+
                                                    position_w_node_schedule.index.minute+
                                                    (position_w_node_schedule.index.second)/60) - 
                                                    ((position_w_node_schedule.loc[:,'trip_start_time'].dt.hour)*60+
                                                    position_w_node_schedule.loc[:,'trip_start_time'].dt.minute+
                                                    (position_w_node_schedule.loc[:,'trip_start_time'].dt.second)/60))

    return position_w_node_schedule

# could be done in parallel!

In [31]:
start = time.time()
unpacked_positions_full_w_start = join_tripstart_unpacked(unpacked_positions_full, full_trip_stop_schedule)
end = time.time()
print(end - start)

84.78130006790161


In [32]:
unpacked_positions_full_w_start[['month_day_trip_veh', 
                                 'vehicle_lat',
       'vehicle_long',
       'shape_dist_traveled',
       'time_from_scheduled_start']].head(10)

,month_day_trip_veh,vehicle_lat,vehicle_long,shape_dist_traveled,time_from_scheduled_start
time_pct,,,,,
2018-08-31 16:59:07-07:00,2018_8_31_39512518_6046,47.723824,-122.344978,46768.3,41.116667
2018-08-31 16:59:36-07:00,2018_8_31_39512547_6071,47.698055,-122.344612,37476.0,35.600000
2018-08-31 16:59:39-07:00,2018_8_31_39512516_6040,47.604149,-122.332672,1105.0,2.650000
2018-08-31 16:59:39-07:00,2018_8_31_39512611_2803,47.712730,-122.344818,42836.9,52.650000
2018-08-31 16:59:52-07:00,2018_8_31_39512539_6061,47.604164,-122.332687,1105.0,7.866667
2018-08-31 16:59:52-07:00,2018_8_31_39512446_6202,47.727604,-122.345016,48252.6,47.866667
2018-08-31 16:59:52-07:00,2018_8_31_39512384_6065,47.661789,-122.347275,23747.2,20.866667
2018-08-31 17:00:23-07:00,2018_8_31_39512518_6046,47.727558,-122.345016,48252.6,42.383333
2018-08-31 17:00:50-07:00,2018_8_31_39512519_6064,47.614998,-122.345284,6183.8,16.833333


In [34]:
unpacked_positions_full_w_start.columns

Index(['route_id', 'timestamp', 'trip_id', 'vehicle_id', 'vehicle_lat',
       'vehicle_long', 'time_utc', 'day', 'year', 'month', 'hour', 'dow',
       'direction_id_x', 'shape_id', 'month_day_trip_veh', 'geometry',
       'near_node_pt', 'shape_pt_sequence', 'dist_to_nearest_route_pt',
       'arrival_time', 'departure_time', 'stop_id', 'stop_sequence',
       'stop_headsign', 'pickup_type', 'drop_off_type', 'shape_dist_traveled',
       'start_gtfs_date', 'end_gtfs_date', 'stop_code', 'stop_name',
       'stop_desc', 'stop_lat', 'stop_lon', 'zone_id', 'stop_url',
       'location_type', 'parent_station', 'stop_timezone', 'trip_start_time',
       'direction_id_y', 'trip_headsign', 'route_short_name', 'route_desc',
       'time_from_scheduled_start'],
      dtype='object')

# export

In [36]:
unpacked_positions_full_w_start.to_csv('route_{}_shape_{}_stopsonly_2018-08-15_2018-12-11.csv'.format(
                                        input_dict['route_id'], input_dict['shape_id']), index=False)